In [ ]:
import albumentations as A
import os
import cv2
import math
import shutil
import os

In [ ]:
IMG_WIDTH, IMG_HEIGHT  = 224, 224
input_folder = '/kaggle/input/dataset'
output_folder = '/kaggle/working/'

In [ ]:
# To make the data balanced. Create 100 examples for each character.

AUG_IMG_NUM = {}

for folder in os.listdir(input_folder):
    AUG_IMG_NUM.update({f'{folder}': math.ceil(100 / 
                                               len(os.listdir(rf'{input_folder}\{folder}')))})

In [ ]:
transform = A.Compose(
    [
        A.Resize(width=IMG_WIDTH, height=IMG_HEIGHT),
        A.Rotate(limit=10, p=0.9),
        A.RGBShift(25, 25, 25, p=0.5),
        A.Blur(blur_limit=3, p=0.6),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9),
        A.RandomShadow(p=0.5),
        A.RandomFog(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9),
        A.RandomRain(p=0.5),
    ])

In [ ]:
for folder in os.listdir(input_folder):
    folder_path = os.path.join(input_folder, folder)
    out_folder_path = os.path.join(output_folder, folder)

    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Create the output folder if it doesn't exist
        os.makedirs(out_folder_path, exist_ok=True)

        for im in os.listdir(folder_path):
            im_path = os.path.join(folder_path, im)
            #print(f"Processing image: {im_path}")
            image = cv2.imread(im_path)

            if image is not None:
                for i in range(AUG_IMG_NUM[folder]):
                    augmented = transform(image=image)
                    augmented_image = augmented['image']

                    out_path = os.path.join(out_folder_path, f"{i}_{im}")
                    cv2.imwrite(out_path, augmented_image)
            else:
                print(f"Failed to read image: {im_path}")

In [ ]:
# make sure sizes are ok

num = 0 # number of images

for folder in os.listdir(output_folder):
    for im in os.listdir(os.path.join(output_folder, folder)):
        num += 1
        im_path = os.path.join(output_folder, folder, im)
        image = cv2.imread(im_path)
        
        if image.shape[:2] != (224, 224):
            print('wroing size found {im_path}')
            break
else:
    print('All ok')
    
print(num)

# Save the working directory

In [ ]:
shutil.make_archive('aug_images', 'zip', '/kaggle/working/')

# Delete all the files in the working directory

In [ ]:
working_directory = '/kaggle/working'

for folder in os.listdir(working_directory):
    folder_path = os.path.join(working_directory, folder)

    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)

print("All folders in the working directory deleted.")